In [1]:
from chan_modeling.archiving import MyThreads, clean, train, analyze, MySentences
from chan_modeling.lda_modeling import train_lda_model, prepare_dictionary, get_topics_for_document
from chan_modeling.functional_helpers import nth
import gensim
from gensim.models.ldamodel import LdaModel
import pprint
from collections import Counter
from operator import itemgetter

Slow version of gensim.models.doc2vec is being used


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
train(50, 4, fname='test_model', dir_name='biz_archive')

2018-08-03 22:13:48,634 : WARNING : Slow version of gensim.models.word2vec is being used
2018-08-03 22:13:48,639 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-08-03 22:13:48,641 : INFO : collecting all words and their counts
2018-08-03 22:13:51,299 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2018-08-03 22:13:51,303 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-08-03 22:13:53,508 : INFO : PROGRESS: at sentence #10000, processed 385521 words, keeping 44 word types
2018-08-03 22:13:56,086 : INFO : PROGRESS: at sentence #20000, processed 813948 words, keeping 60 word types
2018-08-03 22:13:58,256 : INFO : PROGRESS: at sentence #30000, processed 1218595 words, keeping 78 word types
2018-08-03 22:14:00,737 : INFO : PROGRESS: at sentence #40000, processed 1622361 words, keeping 106 word types
2018-08-03 22:14:02,885 : INFO : PRO

KeyboardInterrupt: 

In [4]:
analyze(50, 5, vector_fname='test_model_vectors', test_file_name='test_file.txt')

2018-08-03 21:37:02,053 : INFO : loading KeyedVectors object from test_model_vectors
2018-08-03 21:37:02,684 : INFO : setting ignored attribute syn0norm to None
2018-08-03 21:37:02,685 : INFO : loaded test_model_vectors
2018-08-03 21:37:02,692 : INFO : precomputing L2-norms of word weight vectors


In [3]:
# phrases = gensim.models.Phrases(threads)

In [4]:
# bigrams = gensim.models.phrases.Phraser(phrases)

In [5]:
# nth(bigrams[threads], 45)

In [6]:
doc_term_matrix, dictionary = prepare_dictionary(threads)

2018-07-23 05:20:33,119 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-07-23 05:21:07,445 : INFO : adding document #10000 to Dictionary(93571 unique tokens: ['nuls', 'status', 'update', 'considering', 'buying']...)
2018-07-23 05:21:36,117 : INFO : adding document #20000 to Dictionary(156199 unique tokens: ['nuls', 'status', 'update', 'considering', 'buying']...)
2018-07-23 05:21:49,536 : INFO : built Dictionary(183094 unique tokens: ['nuls', 'status', 'update', 'considering', 'buying']...) from 24526 documents (total 3231880 corpus positions)


In [ ]:
model = train_lda_model(100, dictionary, doc_term_matrix, 6, 10)

In [8]:
# model = LdaModel.load('models/ldamodel')

In [11]:
freqs = Counter()
get_highest_prob_topics = lambda x: x[1] > .1
first = itemgetter(0)
for thread in threads:
    tops = get_topics_for_document(thread, model, dictionary)
    topics = map(first, list(filter(get_highest_prob_topics, tops)))
    freqs[frozenset(topics)] += 1

In [13]:
second_element = itemgetter(1)
more_than_ten_documents = lambda x: x[1] > 10
more_than_one_topic = lambda x: len(x[0]) > 1
only_one_topic = lambda x: len(x[0]) == 1

key_topics = sorted(freqs.items(), key=second_element, reverse=True)
groups = list(filter(only_one_topic, key_topics))
groups = list(filter(more_than_ten_documents, groups))
pprint.pprint(groups)

[(frozenset({17}), 396),
 (frozenset({56}), 240),
 (frozenset({2}), 226),
 (frozenset({26}), 165),
 (frozenset({52}), 150),
 (frozenset({95}), 58),
 (frozenset({96}), 54),
 (frozenset({57}), 46),
 (frozenset({65}), 42),
 (frozenset({20}), 38),
 (frozenset({8}), 26),
 (frozenset({67}), 25),
 (frozenset({89}), 21),
 (frozenset({88}), 20),
 (frozenset({23}), 19),
 (frozenset({31}), 17),
 (frozenset({39}), 16),
 (frozenset({53}), 15),
 (frozenset({76}), 14),
 (frozenset({54}), 13),
 (frozenset({9}), 12),
 (frozenset({34}), 11),
 (frozenset({50}), 11),
 (frozenset({86}), 11),
 (frozenset({51}), 11),
 (frozenset({75}), 11)]


In [14]:
group_sets, _ = zip(*groups)
for grp in group_sets:
    print("="*30)    
    print(list(grp))
    for topic in zip(grp, map(model.show_topic, grp)):
        pprint.pprint(list(topic))
        print('\n')    

[17]
[17,
 [('btc', 0.029735321736004114),
  ('bch', 0.018235823450358846),
  ('going', 0.013776596735470992),
  ('fucking', 0.010818884225922205),
  ('bitcoin', 0.0095549130907119709),
  ('back', 0.0091081876417896043),
  ('go', 0.0085936161539116873),
  ('like', 0.0084320760866134926),
  ('fuck', 0.0078037559696356408),
  ('shit', 0.0075763207976318738)]]


[56]
[56,
 [('coin', 0.028034497569913053),
  ('moon', 0.016117897037794586),
  ('buy', 0.015017074082525554),
  ('im', 0.012590473691372782),
  ('going', 0.012421577701381407),
  ('get', 0.010448415553069707),
  ('like', 0.010434427390919336),
  ('good', 0.0094833308156218477),
  ('next', 0.0088402865556007817),
  ('eth', 0.0087181601285278854)]]


[2]
[2,
 [('get', 0.014059396868597262),
  ('like', 0.013046525369640703),
  ('make', 0.012511899133298872),
  ('money', 0.011190531505368638),
  ('people', 0.010223383373218478),
  ('im', 0.010194057903759305),
  ('want', 0.00861821638711429),
  ('know', 0.0081392533589371378),
  ('fu

In [ ]:
for topic in zip(topics, map(model.show_topic, topics)):
    print(topic)
    print('\n')

In [ ]:
model.top_topics(dictionary=dictionary, corpus=doc_term_matrix)